In [1]:
from quantrocket.license import set_license
set_license("cadba980-de2e-11ee-93e1-4b30e8840aaa")

{'licensekey': 'cadb........0aaa', 'software_license': {}}

In [2]:
from quantrocket.history import create_usstock_db
create_usstock_db("usstock-free-1d", free=True)
{'status': 'successfully created quantrocket.v2.history.usstock-free-1d.sqlite'}

{'status': 'successfully created quantrocket.v2.history.usstock-free-1d.sqlite'}

In [3]:
from quantrocket.history import collect_history
collect_history("usstock-free-1d")
{'status': 'the historical data will be collected asynchronously'}

{'status': 'the historical data will be collected asynchronously'}

In [4]:
from quantrocket.history import download_history_file
download_history_file("usstock-free-1d",
                          start_date="2023-01-01", #YYYY-MM-DD
                          end_date="2023-12-31",
                          fields=["Close"],
                          sids=["FIBBG000B9XRY4"], ##AAPL's sid
                          filepath_or_buffer="apple.csv"
                     )

In [13]:
import csv

with open('apple.csv', newline='') as csvfile:
    # Create a CSV reader object with a dictionary interface
    csvreader = csv.DictReader(csvfile)

    # Iterate over each row in the CSV file
    for row in csvreader:
        # Access data by column names (keys)
        if(row['Date'] == '2023-02-24'):
           print(row['Date'], row['Close'])


2023-02-24 145.9311


In [ ]:
date = input("Enter a date in the format(2023-MM-DD):")
res = date.split('-')




In [22]:
import csv

with open('apple.csv', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    
    prev_row = None
    prev_row1 = None
   
    for row in csvreader:
        if row['Date'] == '2023-11-20':
            if prev_row is not None and prev_row1 is not None:
                print("Previous Row:", prev_row1)
                print("Previous Row:", prev_row)
                print("Current Row:", row)
        prev_row1 = prev_row
        prev_row = row


Previous Row: {'Sid': 'FIBBG000B9XRY4', 'Date': '2023-11-16', 'Close': '189.4682'}
Previous Row: {'Sid': 'FIBBG000B9XRY4', 'Date': '2023-11-17', 'Close': '189.4483'}
Current Row: {'Sid': 'FIBBG000B9XRY4', 'Date': '2023-11-20', 'Close': '191.206'}


In [25]:
import csv

with open('apple.csv', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    close = []
    for row in csvreader:
        close.append(float(row['Close']))
   # print(close)


In [36]:
r = []
for i in range(1,len(close)):
    R = (close[i]-close[i-1])/close[i-1]
    r.append(R*100)
#print(r)
    

In [37]:
s = []
for i in r:
    if(i>=0.1):
        s.append(1)
    elif(i>-0.1 and i<0.1):
        s.append(0)
    else:
        s.append(-1)
#print(s)
#print(len(s))

In [40]:
v = [0]
for i in range(1,len(s)):
    if(s[i-1] == 0 and s[i] == 1):
        v.append(v[i-1]+1)
    elif(s[i-1] == 0 and s[i] == -1 ):
         v.append(v[i-1]-1)
    else:
         v.append(v[i-1])
#print(v)

In [41]:
import pandas as pd

# Step 1: Pull daily close prices for AAPL for the year 2023
# Note: Replace 'path_to_data' with the actual path to the data file containing AAPL prices for 2023
df = pd.read_csv('apple.csv')

# Step 2: Calculate daily returns
df['Return'] = df['Close'].pct_change()

# Step 3: Classify states based on defined logic
def classify_state(return_value):
    if return_value >= 0.1:
        return 1
    elif return_value > -0.1:
        return 0
    else:
        return -1

df['State'] = df['Return'].apply(classify_state)

# Step 4: Determine value function V(d+1)
def calculate_value_function(row):
    if row['State'] == 1 and row['State'].shift(1) == 0:
        return row['V'] + 1
    elif row['State'] == -1 and row['State'].shift(1) == 0:
        return row['V'] - 1
    else:
        return row['V']

df['V'] = 0  # Initial portfolio value
df['V'] = df.apply(calculate_value_function, axis=1)

# Step 5: Calculate transition distribution
transition_matrix = df.groupby(['State', 'State.shift(1)']).size().unstack()
transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)

# Step 6: Implement decision-making process
# Identify optimal points to place buy orders based on the calculated V(N)
optimal_buy_indices = df[df['V'] == df['V'].max()].index.tolist()
max_portfolio_value = df['V'].max()

# Output results
print("Max Portfolio Value:", max_portfolio_value)
print("Optimal Buy Indices:", optimal_buy_indices)

# Save the transition matrix and results to a file or publish them to a GitHub repository
transition_matrix.to_csv('transition_matrix.csv')


AttributeError: 'int' object has no attribute 'shift'